In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Average, Input, Concatenate, GlobalAveragePooling2D, Multiply, Add, Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

In [ ]:
train_dir = 'cotton-disease-dataset/Cotton Disease/train'
val_dir = 'cotton-disease-dataset/Cotton Disease/val'
test_dir = 'cotton-disease-dataset/Cotton Disease/test'

In [ ]:
def preprocess_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    # Apply Gaussian filter
    image_gaussian = cv2.GaussianBlur(image, (5, 5), 0)
    # Apply Laplacian filter
    image_laplacian = cv2.Laplacian(image, cv2.CV_64F)
    image_laplacian = cv2.convertScaleAbs(image_laplacian)
    # Combine original, Gaussian, and Laplacian images
    combined_image = np.concatenate((image, image_gaussian, image_laplacian), axis=-1)
    return combined_image

In [ ]:
def preprocess_dataset(directory):
    datagen = ImageDataGenerator(
        preprocessing_function=preprocess_image,
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

In [ ]:
return datagen.flow_from_directory(
        directory,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

In [ ]:
train_generator = preprocess_dataset(train_dir)
val_generator = preprocess_dataset(val_dir)
test_generator = preprocess_dataset(test_dir)

In [ ]:
print("Train Class Indices: ", train_generator.class_indices)
print("Validation Class Indices: ", val_generator.class_indices)
print("Test Class Indices: ", test_generator.class_indices)

In [ ]:
assert train_generator.class_indices == val_generator.class_indices == test_generator.class_indices

In [ ]:
num_classes = len(train_generator.class_indices)

In [ ]:
def channel_attention(input_feature, ratio=8):
    channel = input_feature.shape[-1]
    shared_layer_one = Dense(channel // ratio, activation='relu', kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')
    shared_layer_two = Dense(channel, kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')

In [ ]:
avg_pool = GlobalAveragePooling2D()(input_feature)
    avg_pool = tf.expand_dims(avg_pool, 1)
    avg_pool = tf.expand_dims(avg_pool, 1)
    avg_pool = shared_layer_one(avg_pool)
    avg_pool = shared_layer_two(avg_pool)

In [ ]:
max_pool = GlobalAveragePooling2D()(input_feature)
    max_pool = tf.expand_dims(max_pool, 1)
    max_pool = tf.expand_dims(max_pool, 1)
    max_pool = shared_layer_one(max_pool)
    max_pool = shared_layer_two(max_pool)

In [ ]:
cbam_feature = Add()([avg_pool, max_pool])
    cbam_feature = tf.keras.activations.sigmoid(cbam_feature)

In [ ]:
return Multiply()([input_feature, cbam_feature])

In [ ]:
def spatial_attention(input_feature):
    kernel_size = 7
    avg_pool = tf.reduce_mean(input_feature, axis=-1, keepdims=True)
    max_pool = tf.reduce_max(input_feature, axis=-1, keepdims=True)
    concat = Concatenate(axis=-1)([avg_pool, max_pool])
    cbam_feature = Conv2D(filters=1, kernel_size=kernel_size, strides=1, padding='same', activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(concat)
    return Multiply()([input_feature, cbam_feature])

In [ ]:
def create_model_with_attention(base_model_class, input_shape=(224, 224, 9), num_classes=num_classes):
    input_layer = Input(shape=input_shape)
    base_model = base_model_class(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
    feature = base_model(input_layer[:, :, :, :3])
    feature_gaussian = base_model(input_layer[:, :, :, 3:6])
    feature_laplacian = base_model(input_layer[:, :, :, 6:])

In [ ]:
concatenated_features = Concatenate()([feature, feature_gaussian, feature_laplacian])

In [ ]:
    feature_with_attention = channel_attention(concatenated_features)
    feature_with_attention = spatial_attention(feature_with_attention)

In [ ]:
x = Flatten()(feature_with_attention)
    x = Dense(228, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

In [ ]:
model = Model(inputs=input_layer, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    return model

In [ ]:
vgg16_model = create_model_with_attention(VGG16)
resnet50_model = create_model_with_attention(ResNet50)
inceptionv3_model = create_model_with_attention(InceptionV3)

In [ ]:
vgg16_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
resnet50_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
inceptionv3_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
true_classes = test_generator.classes

In [ ]:
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(true_classes),
    y=true_classes
)
class_weights = {i: class_weights[i] for i in range(len(class_weights))}

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
history_vgg16 = vgg16_model.fit(
    train_generator,
    epochs=100,
    validation_data=val_generator,
    callbacks=[early_stopping],
    class_weight=class_weights
)

In [ ]:
history_resnet50 = resnet50_model.fit(
    train_generator,
    epochs=100,
    validation_data=val_generator,
    callbacks=[early_stopping],
    class_weight=class_weights
)

In [ ]:
history_inceptionv3 = inceptionv3_model.fit(
    train_generator,
    epochs=100,
    validation_data=val_generator,
    callbacks=[early_stopping],
    class_weight=class_weights
)

In [ ]:
models = [vgg16_model, resnet50_model, inceptionv3_model]
model_inputs = [model.input for model in models]
model_outputs = [model.output for model in models]
ensemble_output = Average()(model_outputs)
ensemble_model = Model(inputs=model_inputs, outputs=ensemble_output)

In [ ]:
ensemble_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
test_loss_vgg16, test_acc_vgg16 = vgg16_model.evaluate(test_generator)
test_loss_resnet50, test_acc_resnet50 = resnet50_model.evaluate(test_generator)
test_loss_inceptionv3, test_acc_inceptionv3 = inceptionv3_model.evaluate(test_generator)

In [ ]:
print(f'Test accuracy VGG16: {test_acc_vgg16}')
print(f'Test accuracy ResNet50: {test_acc_resnet50}')
print(f'Test accuracy InceptionV3: {test_acc_inceptionv3}')

In [ ]:
predictions_vgg16 = vgg16_model.predict(test_generator)
predictions_resnet50 = resnet50_model.predict(test_generator)
predictions_inceptionv3 = inceptionv3_model.predict(test_generator)

In [ ]:
ensemble_predictions = (predictions_vgg16 + predictions_resnet50 + predictions_inceptionv3) / 3
predicted_classes = np.argmax(ensemble_predictions, axis=1)

In [ ]:
class_labels = list(test_generator.class_indices.keys())

In [ ]:
print(classification_report(predicted_classes, true_classes, target_names=class_labels))

In [ ]:
conf_matrix = confusion_matrix(true_classes, predicted_classes)

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.save("Confusion Matrix.Jpg")
plt.show()